In [28]:
%pip install spacy sentence-transformers scikit-learn
!python -m spacy download ru_core_news_sm


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 1.9 MB/s  0:00:08m0:00:0100:01

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


In [29]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""

import spacy

try:
    nlp = spacy.load('ru_core_news_sm', disable=['ner', 'tagger', 'lemmatizer', 'attribute_ruler'])
except OSError:
    from spacy.cli import download
    download('ru_core_news_sm')
    nlp = spacy.load('ru_core_news_sm', disable=['ner', 'tagger', 'lemmatizer', 'attribute_ruler'])

In [30]:
with open('/home/roman/Downloads/Doner_Loren_Furi_(LP)_Litmir.net_bid224439_2a09f.txt', 'r', encoding='cp1251') as f:
    text_a = f.read()

with open('/home/roman/Downloads/avidreaders.ru__rasskazy7.txt', 'r', encoding='utf-8') as f:
    text_b = f.read()

doc_a = nlp(text_a)
doc_b = nlp(text_b)

my_sents = [s.text.strip() for s in doc_a.sents if len(s.text.strip()) > 10][:1800]
other_sents = [s.text.strip() for s in doc_b.sents if len(s.text.strip()) > 10][:1800]

In [31]:
from sklearn.model_selection import train_test_split

texts = my_sents + other_sents
labels = [0] * len(my_sents) + [1] * len(other_sents)

X_train, X_test, y_train, y_test = train_test_split(
    texts, labels,
    test_size=0.2,
    random_state=42,
    stratify=labels
)

print(f"Train: {len(X_train)} примеров")
print(f"Test:  {len(X_test)} примеров")

Train: 2880 примеров
Test:  720 примеров


In [32]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('cointegrated/rubert-tiny2', device="cpu")

X_train_emb = model.encode(X_train, show_progress_bar=True, device="cpu")
X_test_emb = model.encode(X_test, show_progress_bar=True, device="cpu")

print(f"Готово! Размерность вектора: {X_train_emb.shape[1]}")

Batches:   0%|          | 0/90 [00:00<?, ?it/s]

Batches:   0%|          | 0/23 [00:00<?, ?it/s]

Готово! Размерность вектора: 312


In [34]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

clf = LogisticRegression(solver='liblinear', random_state=42, max_iter=100)
clf.fit(X_train_emb, y_train)

y_pred = clf.predict(X_test_emb)

print("\n--- Метрики классификации ---")
print(classification_report(y_test, y_pred, target_names=['Моя 18+ книга', 'Книга Прилепина']))


--- Метрики классификации ---
                 precision    recall  f1-score   support

  Моя 18+ книга       0.92      0.89      0.91       360
Книга Прилепина       0.89      0.93      0.91       360

       accuracy                           0.91       720
      macro avg       0.91      0.91      0.91       720
   weighted avg       0.91      0.91      0.91       720



 Точность 91% на тестовой выборке из 720 предложений. 

Когда модель говорит «это моя книга», она права в 92% случаев.  - precision
Из 360 преддложений моей книги нейронка верно угадала 89%, 320 штук

F1 - метрика, сочетающая пресижн и реколл. 

accuracy, как я понял, метрика, обозначающая отношение правильных угадываний ко всему количеству
macro avg это среднее между пресижнами, реколлами.
weighted avg проверяет на сбалансированность количества выборок